# Initial cleaning
- Rename columns to remove spaces from the name and make them all lowercase.
- Look for the Na-values. Fix them if possible, using a couple of techniques before deciding which is best.
- Look for (ordered) categoricals and label them as such.

In [1]:
# Import
import pyarrow
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import dates
import seaborn as sns

In [ ]:
# Change the file type to .parquet because aws had issues loading the original .csv file
temp = pd.read_csv('data/price_paid_records.csv')
temp.to_parquet('data/price_paid_records.parquet', engine='pyarrow')

In [ ]:
# Load and short information about the columns
df = pd.read_parquet('data/price_paid_records.parquet', engine='pyarrow')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22489348 entries, 0 to 22489347
Data columns (total 11 columns):
 #   Column                             Dtype 
---  ------                             ----- 
 0   Transaction unique identifier      object
 1   Price                              int64 
 2   Date of Transfer                   object
 3   Property Type                      object
 4   Old/New                            object
 5   Duration                           object
 6   Town/City                          object
 7   District                           object
 8   County                             object
 9   PPDCategory Type                   object
 10  Record Status - monthly file only  object
dtypes: int64(1), object(10)
memory usage: 1.8+ GB


# Renaming columns
Making all column names lowercase
Replacing unwanted characters and names:
- ' ' -> '_'
- '/' -> '_'
- 'town_or_city' -> 'city'
- 'old_new' -> 'is_new'

In [6]:
# Creating new column names based on the old ones
new_columns = [col.lower().replace(' ', '_').replace('/', '_') for col in df.columns]
new_columns = ['city' if col=='town_city' else col for col in new_columns]
new_columns = ['is_new' if col=='old_new' else col for col in new_columns]
df.columns = new_columns

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22489348 entries, 0 to 22489347
Data columns (total 11 columns):
 #   Column                             Dtype 
---  ------                             ----- 
 0   transaction_unique_identifier      object
 1   price                              int64 
 2   date_of_transfer                   object
 3   property_type                      object
 4   is_new                             object
 5   duration                           object
 6   city                               object
 7   district                           object
 8   county                             object
 9   ppdcategory_type                   object
 10  record_status_-_monthly_file_only  object
dtypes: int64(1), object(10)
memory usage: 1.8+ GB


How many null values are in each column

In [ ]:
print(df.isnull().sum())

transaction_unique_identifier        0
price                                0
date_of_transfer                     0
property_type                        0
is_new                               0
duration                             0
city                                 0
district                             0
county                               0
ppdcategory_type                     0
record_status_-_monthly_file_only    0
dtype: int64


# Dealing with Categorical data
- Removing 'transaction_unique_identifier'
- Converting 'date_of_transfer' from object dtype to datetime
- Changing 'is_new' from object dtype to bool
- Removing 'record_status_-_monthly_file_only'

In [8]:
# Inspect first 5 entries
df.head()

,transaction_unique_identifier,price,date_of_transfer,property_type,is_new,duration,city,district,county,ppdcategory_type,record_status_-_monthly_file_only
0,{81B82214-7FBC-4129-9F6B-4956B4A663AD},25000,1995-08-18 00:00,T,N,F,OLDHAM,OLDHAM,GREATER MANCHESTER,A,A
1,{8046EC72-1466-42D6-A753-4956BF7CD8A2},42500,1995-08-09 00:00,S,N,F,GRAYS,THURROCK,THURROCK,A,A
2,{278D581A-5BF3-4FCE-AF62-4956D87691E6},45000,1995-06-30 00:00,T,N,F,HIGHBRIDGE,SEDGEMOOR,SOMERSET,A,A
3,{1D861C06-A416-4865-973C-4956DB12CD12},43150,1995-11-24 00:00,T,N,F,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,A,A
4,{DD8645FD-A815-43A6-A7BA-4956E58F1874},18899,1995-06-23 00:00,S,N,F,WAKEFIELD,LEEDS,WEST YORKSHIRE,A,A


Since 'transaction_unique_identifier' is a reference number which is generated automatically when recording each published sale.

This column is useless to since there is no meaning full data stored in the identifier.

In [ ]:
df.drop('transaction_unique_identifier', axis=1, inplace=True)

Converting 'date_of_transfer' from object dtype to datetime

In [ ]:
df['date_of_transfer'] = pd.to_datetime(df['date_of_transfer'])

Changing 'is_new' from object dtype to bool

'is_new' has 2 possible values:
- Y = a newly built property
- N = an established residential building

Mapping 'Y' to 'True' and 'N' to 'False'

In [ ]:
df['is_new'] = df['is_new'].map({'Y': True, 'N': False})

We can see that 'city', 'district', and 'county' detailed information about the location.

Which makes a lot of categories.

But the remaining object columns are truely categorical.

In [ ]:
object_columns = df.select_dtypes(include=['object']).drop(columns=['city', 'district', 'county'])

for col in object_columns.columns:
    unique_values = object_columns[col].unique()
    print(f"Column name '{col}':")
    print(unique_values,"\n")

Column name 'property_type':
['T' 'S' 'D' 'F' 'O'] 

Column name 'duration':
['F' 'L' 'U'] 

Column name 'ppdcategory_type':
['A' 'B'] 

Column name 'record_status_-_monthly_file_only':
['A'] 



From this, we see that column 'record_status_-_monthly_file_only' is useless since all columns have the same value.
Indicates additions, changes and deletions to the records.
This is represented by 3 possible values (A, C, D).
But the provided dataset contains only 'A'.

In [ ]:
df.drop('record_status_-_monthly_file_only', axis=1, inplace=True)

In [14]:
# Short information about the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22489348 entries, 0 to 22489347
Data columns (total 9 columns):
 #   Column            Dtype         
---  ------            -----         
 0   price             int64         
 1   date_of_transfer  datetime64[ns]
 2   property_type     object        
 3   is_new            bool          
 4   duration          object        
 5   city              object        
 6   district          object        
 7   county            object        
 8   ppdcategory_type  object        
dtypes: bool(1), datetime64[ns](1), int64(1), object(6)
memory usage: 1.4+ GB


In [15]:
# Inspect first 5 entries
df.head()

,price,date_of_transfer,property_type,is_new,duration,city,district,county,ppdcategory_type
0,25000,1995-08-18,T,False,F,OLDHAM,OLDHAM,GREATER MANCHESTER,A
1,42500,1995-08-09,S,False,F,GRAYS,THURROCK,THURROCK,A
2,45000,1995-06-30,T,False,F,HIGHBRIDGE,SEDGEMOOR,SOMERSET,A
3,43150,1995-11-24,T,False,F,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,A
4,18899,1995-06-23,S,False,F,WAKEFIELD,LEEDS,WEST YORKSHIRE,A


In [16]:
# Save data in parquet format
df.to_parquet('data/initial_cleaning.parquet', engine='pyarrow')